In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
import csv
with open('nyc_restaurant_inspections.csv', 'r') as fi:
    reader = csv.reader(fi)
    print(next(reader))

['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE', 'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'VIOLATION CODE', 'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE', 'GRADE DATE', 'RECORD DATE', 'INSPECTION TYPE']


In [7]:
def cuisine_camis(pid, record):
    import csv
    if pid==0:
        next(record)
    reader = csv.reader(record)
    for row in reader:
        camis = row[0]
        cuisine = row[7]
        yield (camis, cuisine)
            
rdd = sc.textFile('nyc_restaurant_inspections.csv').cache()

In [18]:
counts = rdd.mapPartitionsWithIndex(cuisine_camis) \
            .map(lambda x: (x, 1)) \
            .reduceByKey(lambda x,y: 1) \
            .map(lambda x: (x[0][1], 1)) \
            .reduceByKey(lambda x,y: x+y) \
            .top(25, key=lambda x: x[1])

In [19]:
counts

[('American', 6002),
 ('Chinese', 2399),
 ('CafÃ©/Coffee/Tea', 1629),
 ('Other', 1296),
 ('Pizza', 1186),
 ('Italian', 1016),
 ('Mexican', 877),
 ('Japanese', 859),
 ('Latin (Cuban, Dominican, Puerto Rican, South & Central American)', 840),
 ('Bakery', 733),
 ('Caribbean', 671),
 ('Spanish', 644),
 ('Donuts', 537),
 ('Pizza/Italian', 483),
 ('Chicken', 456),
 ('Sandwiches', 406),
 ('Juice, Smoothies, Fruit Salads', 382),
 ('Hamburgers', 378),
 ('Asian', 371),
 ('Ice Cream, Gelato, Yogurt, Ices', 339),
 ('Indian', 332),
 ('Jewish/Kosher', 327),
 ('French', 319),
 ('Delicatessen', 294),
 ('Thai', 286)]